In [6]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

In [10]:
# url definition
url = "https://news.sky.com/"

# Request
r1 = requests.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'lxml')

# News identification
coverpage_news = soup1.find_all('div', class_="sdc-site-tiles__item sdc-site-tile sdc-site-tile--has-link")
len(coverpage_news)

# Empty list for img, titles and content 
lst = []

for n in np.arange(0, len(coverpage_news)):
    # Dictionary for saving in k and v 
    d = {}

    # Getting img url
    img = coverpage_news[n].img['src']
    d['Img-url'] = img

    # Extracting h3 tag
    tag = coverpage_news[n].find('h3', class_="sdc-site-tile__headline")

    # Getting title
    title = tag.find('a').find('span').get_text()
    d['Title'] = title

    # Getting the link of the article
    link = "https://news.sky.com" + tag.find('a', class_='sdc-site-tile__headline-link')['href']

    # Reading the content (it is divided in paragraphs)
    try:
        article = requests.get(link)
    except requests.exceptions.ConnectionError:
        requests.status_code = "Connection refused"
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'lxml')
    body = soup_article.find_all('div', class_='sdc-article-body sdc-article-body--story sdc-article-body--lead')
    try:
        x = body[0].find_all('p')
    except IndexError:
        continue

    # Unifying the paragraphs
    list_paragraphs = []
    for p in np.arange(0, len(x)):
        paragraph = x[p].get_text()
        list_paragraphs.append(paragraph)
        final_article = " ".join(list_paragraphs)

    # Removing special characters
    final_article = re.sub("\\xa0", "", final_article)

    d['Content'] = final_article

    lst.append(d)

In [15]:
lst[0]

{'Img-url': 'https://e3.365dm.com/20/06/768x432/skynews-air-ambulances-scene_5018774.jpg?20200620213348',
 'Title': 'Witness: I saw man walk up to group of friends and start stabbing people in neck',
 'Content': 'A witness has told of how he saw a man walk up to a group of about 10 people and stab some of them in the neck and under the arms in a busy Reading park. Lawrence Wort said the attacker then started running towards him but could not catch him and the perpetrator tried to target another group. Three people are confirmed to have died following the multiple stabbings around 7pm on Saturday in Forbury Gardens in the centre of the town. Mr Wort said he had been around 10 metres from the scene and the attack looked "completely random". He said: "So the park was pretty full, a lot of people sat around drinking with friends when one lone person walked through, suddenly shouted some unintelligible words and went around a large group of around 10, trying to stab them. "He stabbed three 